In [1]:
#import statements
import pandas as pd
import numpy as np
import os
import re
import time
from langdetect import detect  
import ast
from bs4 import BeautifulSoup
import requests, json
from nltk import word_tokenize,pos_tag_sents,WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter
from textblob import TextBlob
from sklearn.svm import SVC  
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
# from polyglot.detect import Detector 
# import cld2
from sklearn.ensemble import GradientBoostingClassifier 

In [2]:
# Reads the emoticon look up table file
header = ['EmoticonSymbol','SentimentScore']
emoticon_data = pd.read_csv('EmoticonLookupTable.txt', delimiter='\t', encoding = 'ISO-8859-1',names=header)

In [3]:
#Writing emoticons to a dictionary
emoji_dict = emoticon_data.groupby('EmoticonSymbol')['SentimentScore'].apply(list).to_dict()

In [4]:
#Getting acronyms and slangs from html page and creating a dictionary
resp = requests.get("http://www.netlingo.com/acronyms.php")
soup = BeautifulSoup(resp.text, "html.parser")
slangdict= {}
key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
  for li in div.findAll('li'):
   for a in li.findAll('a'):
       key =a.text
   value = li.text.split(key)[1]
   slangdict[key]=value

In [5]:
#Removing the "-or-" terms in the dictionary and retaining one acronym
for key,value in slangdict.items():
    if "-or-" in value:
       removestring = re.findall('-or-(.*)',value)
       removestring = ''.join(removestring)
       newvalue = value.replace(removestring,'')
       newvalue = newvalue.replace("-or-",'')
       slangdict[key] = newvalue
    elif "-or" in value:
       removestring = re.findall('-or(.*)',value)
       removestring = ''.join(removestring)
       newvalue = value.replace(removestring,'')
       newvalue = newvalue.replace("-or",'')
       slangdict[key] = newvalue
    

In [6]:
slangdict.pop('!',None)
slangdict.pop('*$',None)
slangdict.pop('**//',None)
slangdict.pop(',!!!!',None)

'Talk to the hand'

In [7]:
key_to_be_replaced = []
for keys in slangdict.keys():
    if " or " in keys:
        key_to_be_replaced.append(keys)

#print(key_to_be_replaced)

for keys in key_to_be_replaced:
    getkeys = keys.split("or")
    for x in getkeys:
        x = x.strip()
        slangdict[x]= slangdict[keys]
    slangdict.pop(keys,None)    
    

In [8]:
file = open("Slangdictionary.txt",'w',encoding='utf-8')
file.write(str(slangdict))
file.close()


In [9]:
#read the training data from the csv file
header = ['label','comment','parent_comment']
data = pd.read_table('train-balanced.csv',
                    sep='\t', 
                   delimiter=',', 
                    names=header,
                    usecols=[0,1,9],
                    dtype={'label':int,'comment':str,'parent_comment':str},
                    keep_default_na=False)

In [10]:
print(data.shape)

(1010826, 3)


In [11]:
#helper function to clean the comments
def comment_clean(user_comment):
    # remove trailing \r and \n    
    user_comment.rstrip('\r\n')
    
    #remove the # from hashtag
    if '#' in user_comment:
        hash_tag = re.search('#',user_comment)
        if hash_tag is not None:
            user_comment = user_comment.replace(hash_tag.group(0),' ')
    #remove the redit tags(r/) from comment
    if 'r/' in user_comment:
        r_tag = re.search('r/',user_comment)
        if r_tag is not None:
            user_comment = user_comment.replace(r_tag.group(0),' ')
    #remove the URL links from comments  
    if 'HTTP' in user_comment:
        # url of the form [link name](http://url)
        url_link = re.search('\[(.*)\(HTTP(.*)\)', user_comment)
        if url_link is not None:
            user_comment = user_comment.replace(url_link.group(0),' ')
    if 'http' in user_comment:
        # url of the form [link name](http://url)
        url_link = re.search('\[(.*)\(http(.*)\)', user_comment)
        if url_link is not None:
            user_comment = user_comment.replace(url_link.group(0),' ')
        else:
            #url of the form http:/
            url_link = re.search('http(.*)', user_comment)
            if url_link is not None:
                user_comment = user_comment.replace(url_link.group(0),' ') 
                
    # remove numbers from comments
    user_comment_not_num = re.sub(r'\d+', '', user_comment)   
    
    # Check if the comment has exactly 2 stars
    if user_comment.count('*')==2:
        boldwords = re.search(r"\*(.*?)\*",user_comment)
        #print(boldwords.group(0))
        # Check if the comments have any other text other than **
        if boldwords.group(0) != "**":
            Wordstocapitalize = re.findall(r"\*(.*?)\*",boldwords.group(0))
            Wordstocapitalize = "".join( Wordstocapitalize)
            # Replace the user comment with capitalized words
            user_comment = user_comment.replace(boldwords.group(0),Wordstocapitalize.upper())
    comment_words = re.sub(r"[^a-zA-Z0-9\s\']","",user_comment)         
    comment_words=comment_words.split()
    for word in comment_words:
        if word.upper() in slangdict.keys():
            user_comment = user_comment.replace(word.upper(),slangdict[word.upper()])
        elif word in slangdict.keys():
            user_comment = user_comment.replace(word,slangdict[word]) 
        
    # replace non english comments with empty string
    
    # replace non english comments with empty string
#     try:
#         isReliable, textBytesFound, details = cld2.detect(user_comment_not_num)
#     except:
#         try_text = ''.join(x for x in user_comment_not_num if x.isprintable())
#         isReliable, textBytesFound, details = cld2.detect(try_text)
#     cld_match = details[0][0]
#     if not (cld_match == 'ENGLISH'):
#         poly_match = Detector(user_comment_not_num, quiet=True).language.name
#         if (poly_match != 'English'):
#             user_comment = ' '               
    return user_comment           

In [12]:
#clean each commentnon_eng_data = []
start_time = time.time()
# data['comment'] = data.comment.apply(comment_clean)
data[['comment','parent_comment']] = data[['comment','parent_comment']].applymap(comment_clean)
# remove data with empty comments
valid_comment = data['comment'] != ' '
data = data[valid_comment]
end_time = time.time()
print("time taken ", end_time-start_time)

time taken  78.49125504493713


In [13]:
#write the cleaned data into a csv file
data.to_csv('clean_data_Cldpoly.csv',
           sep= '|',
           index=False)

In [14]:
#read the training data from the csv file
cleaneddata = pd.read_table('clean_data_Cldpoly.csv',
                    sep='|', 
                   # delimiter=',',
                    usecols=[0,1,2],
                    dtype={'label':int,'comment':str},
                    keep_default_na=False)

In [15]:
# Reading the slang dictionary that is already created
f = open("Slangdictionary.txt","r")
res1=f.read()
f.close()
slangdict = ast.literal_eval(res1)

In [18]:
# Capitalizing the characters with *...* and Replacing the words found in slang dictionary with the full forms

#def preprocessing(user_comment):
    # Check if the comment has exactly 2 stars
    
#     if user_comment.count('*')==2:
#         boldwords = re.search(r"\*(.*?)\*",user_comment)
#         #print(boldwords.group(0))
#         # Check if the comments have any other text other than **
#         if boldwords.group(0) != "**":
#             Wordstocapitalize = re.findall(r"\*(.*?)\*",boldwords.group(0))
#             Wordstocapitalize = "".join( Wordstocapitalize)
#             # Replace the user comment with capitalized words
#             user_comment = user_comment.replace(boldwords.group(0),Wordstocapitalize.upper())
#     comment_words = re.sub(r"[^a-zA-Z0-9\s\']","",user_comment)         
#     comment_words=comment_words.split()
#     for word in comment_words:
#         if word.upper() in slangdict.keys():
#             user_comment = user_comment.replace(word.upper(),slangdict[word.upper()])
#         elif word in slangdict.keys():
#             user_comment = user_comment.replace(word,slangdict[word])            
#     return user_comment


In [19]:
#cleaneddata['comment'] = cleaneddata.comment.apply(preprocessing)

In [20]:
#print(cleaneddata['comment'])

In [16]:
def featureextraction(dataframe, field, func, column_names):
    return pd.concat((
        dataframe,
        dataframe[field].apply(
            lambda cell: pd.Series(func(cell), index=column_names))), axis=1)

In [17]:
## function to get list of emojis in a comment
def find_emoji(text):
    return list(x for x in text.split() if x in emoji_dict.keys() )

In [18]:
# Extracting the features for each comment 
# Punctuation Features and presence of sarcastic symbol
def allfeatures(user_comment):
    if '!' or '.' or '?' in user_comment:
        Numofexclaimations = user_comment.count('!')
        Numofdots = user_comment.count('.')
        Numofquestionmarks = user_comment.count('?')
    else:
        Numofexclaimations = 0
        Numofdots = 0
        Numofquestionmarks = 0
    if '(!)' in user_comment:
        SarcasticSymbol = 1
    else:
        SarcasticSymbol = 0
    sentiments = TextBlob(str(user_comment)).sentiment
    polarity = sentiments.polarity
    subjectivity = sentiments.subjectivity
    numofcapitals = sum(x.isupper() for x in user_comment.split() if len(x) > 1 )
    elist = find_emoji(user_comment)
    pscore =0
    nscore = 0
    for item in elist:
        if (emoji_dict[item][0] == 1):
            pscore += 1
        elif (emoji_dict[item][0] == -1):
            nscore += 1
    return Numofexclaimations,Numofdots,Numofquestionmarks,SarcasticSymbol,polarity,subjectivity,numofcapitals,pscore,nscore

In [19]:
start_time = time.time() 
featureddataset = featureextraction(cleaneddata, 'comment', allfeatures, ['Numofexclaimations', 'Numofdots','Numofquestionmarks','SarcasticSymbol','Polarity', 'Subjectivity','NumofCapitalWords','PositiveEmojiCount','NegativeEmojiCount'])
end_time = time.time() 
print("time taken ", end_time-start_time)

time taken  1182.7921845912933


start_time = time.time()
featureddataset = featureextraction(cleaneddata, 'comment', allfeatures, ['Numofexclaimations', 'Numofdots','Numofquestionmarks','SarcasticSymbol','Polarity', 'Subjectivity','NumofCapitalWords','PositiveEmojiCount','NegativeEmojiCount'])
end_time = time.time()
print("time taken ", end_time-start_time)

In [20]:

start_time = time.time()
txt = cleaneddata['comment'].tolist()
#POS tagging for all the tokens in the sentence
tagged_texts = pos_tag_sents(map(word_tokenize, txt))
end_time = time.time()
cleaneddata['POS'] = tagged_texts
print("time taken ", end_time-start_time)

time taken  587.7420217990875


In [21]:

#helper function to collect number of interjection
def comment_interjection(user_comment):
    count = Counter(tag for word,tag in user_comment)
    return count['UH']

In [22]:
##feature extraction
# number of interjection
featureddataset['interjection']  = cleaneddata.POS.apply(comment_interjection)


In [23]:
def get_parent_sentiment(comment):
    sentiments = TextBlob(str(comment)).sentiment
    polarity = sentiments.polarity
    if polarity >= 0.1:
        return 1
    elif polarity < -0.1:
        return -1
    else:
        return 0

In [24]:
start_time = time.time()
featureddataset['parent_sentiment'] = cleaneddata.parent_comment.apply(get_parent_sentiment)
end_time = time.time()
print("time taken ", end_time-start_time)

time taken  539.8422358036041


In [25]:
# load the Glove embedding into memory
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [26]:
from keras.preprocessing.text import Tokenizer

# Tokenize the comments
Word_tokenizer = Tokenizer()
Word_tokenizer.fit_on_texts(featureddataset['comment'])
# Word_tokenizer.num_words = 100000
vocab_size = len(Word_tokenizer.word_index) + 1
#encode the train tokens to sequence
sequences = Word_tokenizer.texts_to_sequences(featureddataset['comment'])

Using TensorFlow backend.


In [57]:
type(sequences[0])
myarray = np.asarray(sequences[0])
x = np.reshape(sequences[0], (len(sequences[0]),1)).T
print(x)
print(myarray)
print(myarray.T)
cos = 179*5 / (np.sqrt(179*179) * np.sqrt(5*5))
print(cos)

[[ 179 3945    5  179  497]]
[ 179 3945    5  179  497]
[ 179 3945    5  179  497]
1.0


In [27]:
# num_words = 100000
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in Word_tokenizer.word_index.items():
#     if i >= num_words:
#         continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [28]:
embedding_matrix.shape

(182821, 100)

In [29]:
import math
embedding_square ={}
keys = range(vocab_size)
for i in keys:
    embedding = embedding_matrix[i]
    sum_square = 0
    for j in range(len(embedding)):
        values = embedding[j]
        sum_square += values*values
    embedding_square[i] = math.sqrt(sum_square)
# print(embedding_square)

In [31]:
len(embedding_square)

182821

In [46]:
file = open("embedding_square.txt",'w',encoding='utf-8')
file.write(str(embedding_square))
file.close()

In [47]:
#write the cleaned data with features into a csv file
featureddataset.to_csv('clean_data_with features.csv',
           sep= '|',
           index=False)

In [37]:
np.array_equal(embedding_matrix[179] ,embeddings_index.get('nice'))
embedding_square[179]

5.358939569778979

In [74]:
import math
def cosine_similarity(word1,word2,v1,v2):
#     "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxy = 0
#     print(word1,word2)
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
#         sumxx += x*x
#         sumyy += y*y
        sumxy += x*y
#     print(sumxy)
#     print(embedding_square[word1]*embedding_square[word1])
#     cosine = sumxy/(embedding_square[word1]*embedding_square[word2])
#     print(cosine)
#     return cosine
    return sumxy/(embedding_square[word1]*embedding_square[word2])

In [103]:
x = np.matrix([ 0,  1,  2,  15])
# x = matrix([[ 0,  1,  2,  15],
#         [ 4,  12,  6,  7],
#         [ 8,  9, 10, 11]])

# print(x.max(x.max(0)))
# print(x.max(0))
y = x.max(0)
print(y.max())
print(y.min())
# print(x.min(0))
z = x.min(0)
print(z.max())
print(z.min())
# print(x.max(1))
a=x.shape
print(a)
print(x.shape[1])


15
0
15
0
(1, 4)
4


In [75]:
def calculate_similarity(comment_token):
    token_array = np.matrix(comment_token)
    comment_len = token_array.shape[1]
#     print(comment_token[0])
#     print(comment_len)
    most_similar = least_similar = most_dissimilar = least_dissimilar = 0
    if comment_len > 0:
        mat = np.zeros(shape=(comment_len,comment_len))
        for i in range(0,comment_len):
            for j in range(i+1,comment_len):
                a = comment_token[i]
                b = comment_token[j]
                mat[i][j] = cosine_similarity(a,b,embedding_matrix[a],embedding_matrix[b].T)
                mat[j][i] = mat[i][j]
#         print(mat)
    #get the most similar
        similar_mat = mat.max(0)
        if()
        most_similar = similar_mat.max()
        least_similar = similar_mat.min()
        dissimilar_mat = mat.min(0)
        most_dissimilar = dissimilar_mat.max()
        least_dissimilar = dissimilar_mat.min()
        
    return most_similar, least_similar, most_dissimilar, least_dissimilar            

In [73]:
calculate_similarity([179,179])

28.718233312542907
28.71823331254291
0.9999999999999999
[[0. 1.]
 [1. 0.]]


(0.9999999999999999, 0.9999999999999999, 0.0, 0.0)

In [60]:
# test
calculate_similarity([179, 3945, 5, 179, 497])

179
5
179 3945
2.660808595734904
179 5
11.38494467833491
179 179
28.718233312542907
179 497
12.338481814359477
3945 5
-0.07627495684197177
3945 179
2.660808595734904
3945 497
4.106126702458106
5 179
11.38494467833491
5 497
19.017276923175036
179 497
12.338481814359477


(0.9999999999999999, 0.16893828121130872, 0.0, -0.0028069642283230986)

In [37]:
embedding_matrix[179]

array([-0.18554001,  0.047152  ,  0.34867001, -0.23114   , -0.26082999,
        0.63107997,  0.55686998,  0.61622   , -0.15436999, -0.38381001,
        0.12445   , -0.26999   , -0.29196   ,  0.1125    ,  0.36035001,
        0.70688999, -0.33891001, -0.26949999,  0.17481001,  0.97048002,
        0.23014   ,  0.63168001, -0.24542999, -0.72890002,  0.32517001,
       -0.21118   , -0.80353999, -0.59863001, -0.10182   , -0.87826002,
       -0.80162001,  0.20998999,  0.64598   , -0.38238999,  0.64512002,
        0.73045999, -0.19881   ,  0.35716999,  0.19135   , -0.43686   ,
        0.75955999, -0.66430998,  0.34509   , -1.03830004, -0.50490999,
        0.19976   , -0.041208  ,  0.16952001,  0.17821001, -0.84249002,
        0.21991999, -0.16474   , -0.24669001,  0.34117001, -0.59713   ,
       -2.3434    ,  0.31483999,  0.69668001,  0.53368002, -0.62857002,
       -0.197     ,  0.52240998, -1.59029996, -0.16474999,  0.62553   ,
       -0.094116  ,  0.0070705 ,  0.22617   , -0.45697999, -0.53

In [76]:
featureddataset['embedding'] = sequences

In [77]:
featureddataset.head()

,label,comment,parent_comment,Numofexclaimations,Numofdots,Numofquestionmarks,SarcasticSymbol,Polarity,Subjectivity,NumofCapitalWords,PositiveEmojiCount,NegativeEmojiCount,interjection,parent_sentiment,embedding
0,0,Nice Crib and Nice Hand.,"Yeah, I get that argument. At this point, I'd ...",0.0,1.0,0.0,0.0,0.600000,1.000000,0.0,0.0,0.0,0,1,"[179, 3945, 5, 179, 497]"
1,0,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...,0.0,0.0,1.0,0.0,0.392857,0.517857,0.0,0.0,0.0,0,1,"[6, 45, 61, 964, 694, 171, 255, 964, 694, 55, ..."
2,0,"They were underdogs earlier today, but since G...",They're favored to win.,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0,0.0,0,1,"[20, 106, 20010, 1940, 313, 13, 207, 43164, 34..."
3,0,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz,0.0,1.0,0.0,0.0,0.193182,0.727273,0.0,0.0,0.0,0,0,"[14, 1039, 132, 332, 946, 8, 1, 146, 2324, 476..."
4,0,I could use one of those tools.,Yep can confirm I saw the tool they use for th...,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0,"[4, 94, 134, 47, 8, 89, 3439]"


In [78]:
start_time = time.time()
embedded_dataset = featureextraction(featureddataset, 'embedding', calculate_similarity, ['most_similar','least_similar','most_dissimilar','least_dissimilar'])
end_time = time.time()
print("time taken ", end_time-start_time)

time taken  7557.038921833038


In [97]:
values = {'most_similar': 0, 'least_similar': 0, 'most_dissimilar': 0, 'least_dissimilar': 0}
embedded_dataset = embedded_dataset.fillna(value=values)
embedded_dataset = embedded_dataset.drop(columns=['embedding'])

In [98]:
embedded_dataset.head()

,label,comment,parent_comment,Numofexclaimations,Numofdots,Numofquestionmarks,SarcasticSymbol,Polarity,Subjectivity,NumofCapitalWords,PositiveEmojiCount,NegativeEmojiCount,interjection,parent_sentiment,most_similar,least_similar,most_dissimilar,least_dissimilar
0,0,Nice Crib and Nice Hand.,"Yeah, I get that argument. At this point, I'd ...",0.0,1.0,0.0,0.0,0.600000,1.000000,0.0,0.0,0.0,0,1,1.000000,0.168938,0.0,-0.002807
1,0,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...,0.0,0.0,1.0,0.0,0.392857,0.517857,0.0,0.0,0.0,0,1,1.000000,0.583701,0.0,0.000000
2,0,"They were underdogs earlier today, but since G...",They're favored to win.,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0,0.0,0,1,0.000000,0.000000,0.0,0.000000
3,0,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz,0.0,1.0,0.0,0.0,0.193182,0.727273,0.0,0.0,0.0,0,0,0.000000,0.000000,0.0,0.000000
4,0,I could use one of those tools.,Yep can confirm I saw the tool they use for th...,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0,0.777721,0.634564,0.0,0.000000


In [81]:
embedding_matrix[354]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# from numpy import dot
# from numpy.linalg import norm

# cos_sim = dot(a, b)/(norm(a)*norm(b))

            
            

In [99]:
# embedded_dataset.drop(columns=['embedding'])
from sklearn.model_selection import train_test_split
# training_data, test_data = train_test_split(embedded_dataset, test_size=0.20)
training_data, test_data, y_train, y_test = train_test_split(embedded_dataset, embedded_dataset['label'], 
                                                    test_size=0.20)

In [126]:
# embedded_dataset.drop(columns=['embedding'])
print(training_data.shape)

(808624, 18)


In [100]:
newtrain = pd.DataFrame(training_data.iloc[:, 3:])

In [122]:
print(newtrain.shape)

(808624, 15)


In [101]:
targetlabel = training_data.iloc[:,0]

In [121]:
print(targetlabel.shape)

(808624,)


In [102]:
start_time = time.time()
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.5, max_depth=1)
clf.fit(newtrain,targetlabel)
end_time = time.time()
print("time taken ", end_time-start_time)

time taken  44.04390478134155


In [103]:
test_data_new = test_data.iloc[:, 3:] 

In [119]:
print(test_data_new.shape)

(202157, 15)


In [104]:
test_data_target = test_data.iloc[:,0]

In [120]:
print(test_data_target.shape)

(202157,)


In [105]:
predictions = clf.predict(test_data_new)

In [106]:
print("Confusion Matrix")
print(confusion_matrix(test_data_target, predictions))
print("Classification Report")
print(classification_report(test_data_target, predictions))

Confusion Matrix
[[71179 29832]
 [52228 48918]]
Classification Report
              precision    recall  f1-score   support

           0       0.58      0.70      0.63    101011
           1       0.62      0.48      0.54    101146

   micro avg       0.59      0.59      0.59    202157
   macro avg       0.60      0.59      0.59    202157
weighted avg       0.60      0.59      0.59    202157



In [107]:
print(accuracy_score(test_data_target, predictions))

0.5940778701702142


In [122]:
#learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
# for learning_rate in learning_rates:
#     gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
#     gb.fit(newtrain,targetlabel)
#     predictions = clf.predict(test_data_new)
#     print("Confusion Matrix")
#     print(confusion_matrix(test_data_target, predictions))
#     print()
#     print("Classification Report")
#     print(classification_report(test_data_target, predictions))
#     print()
#     print(accuracy_score(test_data_target, predictions))

In [112]:
# Create a list of the feature column's names
features = embedded_dataset.columns[3:]

In [113]:
X_train = training_data[features]
y_train = training_data['label']
X_test = test_data[features]
y_test = test_data['label']

In [114]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest Classifier. 
clf = RandomForestClassifier(n_jobs=2, random_state=0)

In [115]:
# Train the Classifier to take the training features 
start_time = time.time()
clf.fit(X_train, y_train )
end_time = time.time()
print("time taken ", end_time-start_time)

C:\Users\nayan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


time taken  14.977846622467041


In [116]:
# Apply the Classifier we trained to the test data
y_predit = clf.predict(X_test)

In [117]:
print(confusion_matrix(y_test,y_predit))  
print(classification_report(y_test,y_predit))  
print(accuracy_score(y_test,y_predit))

[[69374 31637]
 [51588 49558]]
              precision    recall  f1-score   support

           0       0.57      0.69      0.63    101011
           1       0.61      0.49      0.54    101146

   micro avg       0.59      0.59      0.59    202157
   macro avg       0.59      0.59      0.58    202157
weighted avg       0.59      0.59      0.58    202157

0.588315022482526


In [ ]:
from sklearn.svm import SVC  
start_time = time.time()
# linear kernel
svclassifier = SVC(kernel='linear',C=1,gamma=1)  
svclassifier.fit(X_train, y_train) 

y_pred = svclassifier.predict(X_test)  

end_time = time.time()
print("time taken ", end_time-start_time)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))

In [ ]:
# Guassian kernel
start_time = time.time()
svclassifier = SVC(kernel='rbf',C=1,gamma=1)  
svclassifier.fit(X_train, y_train) 

y_pred = svclassifier.predict(X_test)  
end_time = time.time()
print("time taken ", end_time-start_time)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))

In [ ]:
# polynomial kernel with degree 3
start_time = time.time()
svclassifier = SVC(kernel='poly', degree=3) 
svclassifier.fit(X_train, y_train) 

y_pred = svclassifier.predict(X_test) 
end_time = time.time()
print("time taken ", end_time-start_time)

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))